# analyze the 100 sc items 

- we will look at the 100 datasets
- figure out why we dropped some 
- plot some stats


## conclusion: no idea how i generated the x40 list... so this notebook is shite

In [48]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = True

import matplotlib as mpl
mpl.rc("figure", dpi=200)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
%time 
# load the data
from natto import input 
from natto import process
from ubergauss import tools


names = input.get100names()

def loadandgo(n):
    data = input.load100(n,path = '/home/ubuntu/repos/natto/natto/data')
    gs= process.annotate_genescores(data, mingenes = 200)
    gs.X = np.expm1(gs.X)
    return gs 

annotated = tools.xmap(loadandgo,names)

#data = [input.load100(n,path = '/home/ubuntu/repos/natto/natto/data')  for n in names]
#annotated = [process.annotate_genescores(d, mingenes = 1000) for d in data]  

In [50]:
# so we collect stats and put them in a table 



from matplotlib import pyplot as plt
import numpy as np
from lmz import *
import seaborn as sns 
import pandas as pd 
pd.options.display.max_rows = 1000

data = {'names': names}


# lets say we need 2k annotated cells
data['annotated cells']=[ sum(a.obs['true']!=-1) for a in annotated]
data['diverse cells'] = [ np.sum( np.array([ len(row.indices) for row in a.X]) > 1000 ) for a in annotated]
data['numgoodgenes'] = [ sum(a.varm['scores']!=np.NINF) for a in annotated]
data['reads > 1400'] = [ sum(np.sum(a.X,axis= 1) > 1500 ) for a in annotated]


table = pd.DataFrame(data)
table

,names,annotated cells,diverse cells,numgoodgenes,reads > 1400
0,Adipose_10xchromium_SRA710104-SRS3334253_9399,8923,8148,8768,[[[[[9914]]]]]
1,Adipose_10xchromium_SRA710104-SRS3334254_9107,8653,8333,10040,[[[[[10022]]]]]
2,Adipose_10xchromium_SRA710104-SRS3334255_9510,9035,8732,8586,[[[[[9947]]]]]
3,Bonemarrow_10xchromium_SRA779509-SRS3805245_6248,5904,4212,24331,[[[[[6670]]]]]
4,Bonemarrow_10xchromium_SRA779509-SRS3805246_6024,5589,1173,21813,[[[[[6696]]]]]
5,Bonemarrow_10xchromium_SRA779509-SRS3805247_7623,6524,956,22647,[[[[[8067]]]]]
6,Bonemarrow_10xchromium_SRA779509-SRS3805248_6395,5500,796,21692,[[[[[6538]]]]]
7,Bonemarrow_10xchromium_SRA779509-SRS3805255_7559,7165,2819,22925,[[[[[7927]]]]]
8,Bonemarrow_10xchromium_SRA779509-SRS3805258_5683,5358,2056,22455,[[[[[5942]]]]]
9,Bonemarrow_10xchromium_SRA779509-SRS3805262_6431,6090,4717,25115,[[[[[6789]]]]]


In [ ]:
annotated[0].X[0].data

In [ ]:
if False:
    # another problem was that many cells could have too few reads 
    for name, ann in zip(names, annotated):
        print(name)
        plt.hist(np.sum(ann.X, axis = 1), bins = 75)
        plt.show()

In [ ]:
col = Zip(names, annot)
col = [c for c in col if c[1]>1500 ]
print ([c[0][:5] for c in col])
print(len(col))

# now make sure that 

In [ ]:
# so now we have some tissues with few representatives, lets clean this also


col = [c[0] for c in col] # list of names
a,b = np.unique([c[:5] for c in col ], return_counts=True)
itemcounts = Zip(a,b)
shitlist =  [item for item, count in itemcounts if count < 4]
flist = [c for c in col if c[:5] not in shitlist]


In [ ]:
flist

In [ ]:
import seaborn as sns 
import pandas as pd 

pdata = pd.DataFrame({'name':a,'count':b})
sns.barplot(data  = pdata,x='name',y='count')

In [ ]:

data['reads > 1400'] = [ sum(np.sum(a.X,axis= 1) > 1500 ) for a in annotated]
